## Add GPS Metadata capture for line and polygon layers

This notebook configures an existing line or polygon feature layer to record GPS metadata using ArcGIS Field Maps.

The notebook performs the following actions:

- Adds the esri_fixtype domain
- Adds a series of summary statistics fields
- Enables attachments if needed

Note: you must be the owner of the feature layer or an organization admin user

### Steps

1. Update the following variables
    - `org_url`
    - `username`
    - `item_id`
    - `feature_layer_index`
2. Run all cells


In [5]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

# update this information prior to running
org_url = "https://www.arcgis.com"
username = "doug_nitro"
item_id = "73123abd254045b7a396c497d442fffc" # item_id of feature layer
feature_layer_index = 0 #index of layer you want to update

gis = GIS(org_url, username)

In [6]:
# get the feature layer by ID
item = gis.content.get(item_id)

if (item is None):
    raise TypeError("Cannot Find item")

# check the item type is a feature service
if (item.type != 'Feature Service'):
    raise TypeError("Item is not a feature service")

feature_layer_collection = FeatureLayerCollection.fromitem(item)

feature_layer = feature_layer_collection.layers[feature_layer_index]

# make sure layer is a line or polygon layer
if ((feature_layer.properties.geometryType != 'esriGeometryPolyline') and 
    (feature_layer.properties.geometryType != 'esriGeometryPolygon')):
    raise TypeError("Feature layer is not a line or polygon layer")

# check and enable attachments
if (not feature_layer.properties.hasAttachments):    
    feature_layer.manager.update_definition({"hasAttachments": True})
    print ("Enabled attachments")

In [12]:
# Extract existing fields from Feature layer service definition
feature_layer_fields = feature_layer.properties.fields

# New fields which need to be added
gnss_metadata_fields = {'fields': []}

# Operations list - Add or Update GNSS Metadata fields.
operations = []

# Ensure all fields are present; if not, add them to the definition

# esrignss_avg_h_rms
avg_horizontal_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_avg_h_rms']

if not avg_horizontal_accuracy_field:
    gnss_metadata_fields['fields'].append({'name': 'esrignss_avg_h_rms',
                                            'type': 'esriFieldTypeDouble',
                                            'alias': 'Average horizontal accuracy (m)',
                                            'sqlType': 'sqlTypeOther',
                                            'nullable': True,
                                            'editable': True,
                                            'domain': None,
                                            'defaultValue': None})

# esrignss_avg_v_rms
avg_vertical_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_avg_v_rms']

if not avg_vertical_accuracy_field:
    gnss_metadata_fields['fields'].append({'name': 'esrignss_avg_v_rms',
                                            'type': 'esriFieldTypeDouble',
                                            'alias': 'Average vertical accuracy (m)',
                                            'sqlType': 'sqlTypeOther',
                                            'nullable': True,
                                            'editable': True,
                                            'domain': None,
                                            'defaultValue': None})

# esrignss_worst_h_rms
worst_horizontal_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_worst_h_rms']

if not worst_horizontal_accuracy_field:
    gnss_metadata_fields['fields'].append({'name': 'esrignss_worst_h_rms',
                                            'type': 'esriFieldTypeDouble',
                                            'alias': 'Worst horizontal accuracy (m)',
                                            'sqlType': 'sqlTypeOther',
                                            'nullable': True,
                                            'editable': True,
                                            'domain': None,
                                            'defaultValue': None})

# esrignss_worst_v_rms
worst_vertical_accuracy_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_worst_v_rms']

if not worst_vertical_accuracy_field:
    gnss_metadata_fields['fields'].append({'name': 'esrignss_worst_v_rms',
                                            'type': 'esriFieldTypeDouble',
                                            'alias': 'Worst vertical accuracy (m)',
                                            'sqlType': 'sqlTypeOther',
                                            'nullable': True,
                                            'editable': True,
                                            'domain': None,
                                            'defaultValue': None})


# esrignss_worst_fixtype
worst_fixtype_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_worst_fixtype']

if worst_fixtype_field:
    # Field does exist check if the domain is set.
    if worst_fixtype_field[0]['domain'] is None:
        #if ([operation for operation in operations if operation == 'updateDefinition']):
        operations.append('updateDefinition')

        worst_fixtype_field_index = feature_layer_fields.index(worst_fixtype_field[0])
        fixtype_domain = {'type': 'codedValue',
                            'name': 'esri_fix_type_domain',
                            'codedValues': [{'name': 'Fix not valid',
                                            'code': 0},
                                            {'name': 'GPS', 'code': 1},
                                            {'name': 'Differential GPS', 'code': 2},
                                            {'name': 'RTK Fixed', 'code': 4},
                                            {'name': 'RTK Float', 'code': 5}]}
        feature_layer_fields[worst_fixtype_field_index]['domain'] = fixtype_domain

else:
    gnss_metadata_fields['fields'].append({'name': 'esrignss_worst_fixtype',
                                            'type': 'esriFieldTypeSmallInteger',
                                            'alias': 'Worst fix type',
                                            'sqlType': 'sqlTypeOther',
                                            'nullable': True,
                                            'editable': True,
                                            'domain': {'type': 'codedValue',
                                                    'name': 'ESRI_FIX_TYPE_DOMAIN',
                                                    'codedValues': [
                                                        {'name': 'Fix not valid', 'code': 0},
                                                        {'name': 'GPS', 'code': 1},
                                                        {'name': 'Differential GPS', 'code': 2},
                                                        {'name': 'RTK Fixed', 'code': 4},
                                                        {'name': 'RTK Float', 'code': 5}]},
                                            'defaultValue': None})

# esrignss_manual_locations
num_manual_locations_field = [field for field in feature_layer_fields if field['name'].lower() == 'esrignss_manual_locations']

if not num_manual_locations_field:
    gnss_metadata_fields['fields'].append({'name': 'esrignss_manual_locations',
                                            'type': 'esriFieldTypeInteger',
                                            'alias': 'Number of manual locations',
                                            'sqlType': 'sqlTypeOther',
                                            'nullable': True,
                                            'editable': True,
                                            'domain': None,
                                            'defaultValue': None})

# Check if AddToDefinition operation needs to be added.
initial_featurelayer_fields_count = len(feature_layer_fields)
if (len(gnss_metadata_fields[
            'fields']) + initial_featurelayer_fields_count
        ) > initial_featurelayer_fields_count:
    operations.append('addToDefinition')

In [11]:
# Check if any changes are required
if not operations:
    print("No changes to GPS metadata fields.")
else:
    # Add or update service definition
    for operation in operations:

        # Add
        if operation == 'addToDefinition':
            response = feature_layer.manager.add_to_definition(
                gnss_metadata_fields)
            print("Successfully added GPS metadata fields.")

        # Modify
        else:
            response = feature_layer.manager.update_definition(
                feature_layer_fields)
            print("Successfully updated GPS metadata fields.")

        result = response['success']

        if not result:
            print('Failed to update Feature layer service definition.')
        else:
            print('Service definition updated successfully.')  

No changes to GPS metadata fields.
